In [ ]:
# ================================
# 1. Install Apache Pig
# ================================
!wget https://downloads.apache.org/pig/pig-0.17.0/pig-0.17.0.tar.gz
!tar -xzf pig-0.17.0.tar.gz


!mv pig-0.17.0 /content/pig

import os
os.environ['PIG_HOME'] = '/content/pig'
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['PIG_HOME'], 'bin')

--2025-09-25 04:37:55--  https://downloads.apache.org/pig/pig-0.17.0/pig-0.17.0.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.208.237, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.208.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230606579 (220M) [application/x-gzip]
Saving to: ‘pig-0.17.0.tar.gz.1’

pig-0.17.0.tar.gz.1 100%[===================>] 219.92M  24.5MB/s    in 12s     

2025-09-25 04:38:07 (18.9 MB/s) - ‘pig-0.17.0.tar.gz.1’ saved [230606579/230606579]



In [ ]:
# ================================
# 2. Create Input CSV Files
# ================================

students = """1,Ravi,CSE,85
2,Anita,IT,55
3,John,CSE,72
4,Kiran,ECE,67
5,Meera,IT,90
"""
with open("students.csv", "w") as f:
    f.write(students)

departments = """CSE,Dr.Sharma
IT,Dr.Verma
ECE,Dr.Rao
"""
with open("departments.csv", "w") as f:
    f.write(departments)

In [ ]:
# ================================
# 3. Write the Pig Latin Script
# ================================

pig_script = r"""
-- Load student and department data
students = LOAD 'students.csv' USING PigStorage(',')
           AS (id:int, name:chararray, dept:chararray, marks:int);

departments = LOAD 'departments.csv' USING PigStorage(',')
              AS (dept:chararray, hod:chararray);

-- Filter: students with marks > 60
good_students = FILTER students BY marks > 60;

-- Project: select only name, dept, marks
projected = FOREACH good_students GENERATE name, dept, marks;

-- Group: by department
grouped = GROUP projected BY dept;

-- Sort: order students by marks descending
sorted = ORDER projected BY marks DESC;

-- Join: combine with department HOD info
joined = JOIN projected BY dept, departments BY dept;

-- Dump results
DUMP sorted;
DUMP grouped;
DUMP joined;
"""
with open("program.pig", "w") as f:
    f.write(pig_script)

In [ ]:
# ================================
# 4. Run the Pig Script (Local Mode)
# ================================
#/usr/lib/jvm/java-8-openjdk-amd64
!export JAVA_HOME = /usr/lib/jvm/java-11-openjdk-amd64
!export PATH = $JAVA_HOME/bin:$PATH
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']

!pig -x local program.pig

/bin/bash: line 1: export: `=': not a valid identifier
/bin/bash: line 1: export: `/usr/lib/jvm/java-11-openjdk-amd64': not a valid identifier
/bin/bash: line 1: export: `=': not a valid identifier
/bin/bash: line 1: export: `/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/pig/bin:/content/pig/bin': not a valid identifier
2025-09-25 04:46:15,460 INFO  [main] pig.ExecTypeProvider (ExecTypeProvider.java:selectExecType(41)) - Trying ExecType : LOCAL
2025-09-25 04:46:15,463 INFO  [main] pig.ExecTypeProvider (ExecTypeProvider.java:selectExecType(43)) - Picked LOCAL as the ExecType
2025-09-25 04:46:15,531 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2025-09-25 04:46:15,531 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1758775575524.log
2025-09-25 04:46:15,556 [main] INFO  org.apache.had